In [27]:
import pandas as pd
import numpy as np
import requests
import json

In [28]:
# df = pd.read_csv('../data/dataset_uncensored.csv')
df = pd.read_csv('../data/test_data_final.csv')
# df['start'] = df['start'].apply(lambda x: x-1)
df

,chr,start,end,ref,alt,region_type,reg_id,data_source,id,driver
0,11,111231125,111231125,G,C,enhancers.bed,enhancers::chr11:111222000-111233000::NA::NA,Rheinbay et al 2020,test0,1
1,11,111232885,111232885,C,T,enhancers.bed,enhancers::chr11:111222000-111233000::NA::NA,Rheinbay et al 2020,test1,1
2,7,86866372,86866372,T,C,enhancers.bed,enhancers::chr7:86865600-86866400::NA::NA,Rheinbay et al 2020,test2,1
3,1,2267139,2267139,G,A,enhancers.bed,enhancers::chr1:2267000-2269600::NA::NA,Rheinbay et al 2020,test3,1
4,11,111226848,111226848,T,C,enhancers.bed,enhancers::chr11:111222000-111233000::NA::NA,Rheinbay et al 2020,test4,1
...,...,...,...,...,...,...,...,...,...,...
1057,9,125026995,125026996,C,G,Breast,MRRF,Dr.Nod 2023,test1116,1
1058,9,124049461,124049462,T,A,Breast,MRRF,Dr.Nod 2023,test1117,1
1059,X,70338403,70338404,G,A,Brain,MED12,Dr.Nod 2023,test1118,1
1060,X,70401837,70401838,G,T,Brain,MED12,Dr.Nod 2023,test1119,1


In [29]:
def call_repeatmasker(df):
    res = pd.DataFrame()
    for index, row in df.iterrows():
        print(index)
        if row['temp_start'] >= row['end']:
            url = """https://api.genome.ucsc.edu/getData/track?genome=hg19;track=rmsk;chrom=chr{};start={};end={}""".format(row['chr'], row['start'], row['start'] + 1)
        else:
            url = """https://api.genome.ucsc.edu/getData/track?genome=hg19;track=rmsk;chrom=chr{};start={};end={}""".format(row['chr'], row['start'], row['end'])
        response = requests.get(url)
        res = pd.concat([res, pd.DataFrame(response.json())])
    return res

In [30]:
df1 = call_repeatmasker(df)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [51]:
def post_processing(res, df):
    res = res[['start', 'end', 'chrom', 'rmsk', 'itemsReturned']]
    res['chrom'] = res['chrom'].apply(lambda x: x.replace('chr', ''))
    
    json_struct = json.loads(res.to_json(orient="records"))
    df_flat = pd.io.json.json_normalize(json_struct)
    df_flat.columns = df_flat.columns.str.replace("rmsk.", "")
    df_flat = df_flat.drop_duplicates(keep='first')
    df_flat.reset_index(inplace = True, drop = True)
    df_flat.drop(['id', 'end'], inplace=True, axis=1)
    df_merge = pd.merge(df, df_flat, how='left', left_on=['chr', 'start'], right_on=['chrom', 'start'])
    df_merge = df_merge[~df_merge['chrom'].isna()].reset_index(drop=True,inplace=False)
    # df_merge['start'] = df_merge['start'].apply(lambda x: x+1)
    df_merge.drop_duplicates().reset_index(drop=True)
    return df_merge

In [52]:
res = post_processing(df1, df)
res

C:\Users\Sana\AppData\Local\Temp\ipykernel_29352\2762229421.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res['chrom'] = res['chrom'].apply(lambda x: x.replace('chr', ''))
C:\Users\Sana\AppData\Local\Temp\ipykernel_29352\2762229421.py:6: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  df_flat = pd.io.json.json_normalize(json_struct)
C:\Users\Sana\AppData\Local\Temp\ipykernel_29352\2762229421.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df_flat.columns = df_flat.columns.str.replace("rmsk.", "")


,chr,start,end,ref,alt,region_type,reg_id,data_source,id,driver,...,genoStart,genoEnd,genoLeft,strand,repName,repClass,repFamily,repStart,repEnd,repLeft
0,11,111226848,111226848,T,C,enhancers.bed,enhancers::chr11:111222000-111233000::NA::NA,Rheinbay et al 2020,test4,1,...,111226573.0,111226942.0,-23779574.0,+,Charlie19a,DNA,hAT-Charlie,1.0,382.0,-4.0
1,11,111226707,111226707,G,A,enhancers.bed,enhancers::chr11:111222000-111233000::NA::NA,Rheinbay et al 2020,test6,1,...,111226573.0,111226942.0,-23779574.0,+,Charlie19a,DNA,hAT-Charlie,1.0,382.0,-4.0
2,17,8078565,8078565,G,A,gc19_pc.3utr.bed,gc19_pc.3utr::gencode::TMEM107::ENSG0000017902...,Rheinbay et al 2020,test42,1,...,8078380.0,8078684.0,-73116526.0,-,AluSx,SINE,Alu,-7.0,305.0,1.0
3,17,8076833,8076833,C,A,gc19_pc.3utr.bed,gc19_pc.3utr::gencode::TMEM107::ENSG0000017902...,Rheinbay et al 2020,test47,1,...,8076770.0,8076906.0,-73118304.0,-,U8,snRNA,snRNA,0.0,136.0,1.0
4,17,8076893,8076893,T,A,gc19_pc.3utr.bed,gc19_pc.3utr::gencode::TMEM107::ENSG0000017902...,Rheinbay et al 2020,test48,1,...,8076770.0,8076906.0,-73118304.0,-,U8,snRNA,snRNA,0.0,136.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,6,35699785,35699786,G,C,Brain,RPL10A,Dr.Nod 2023,test1086,1,...,35699729.0,35699792.0,-135415275.0,+,G-rich,Low_complexity,Low_complexity,5.0,68.0,0.0
208,7,48126778,48126779,A,T,Breast,HUS1,Dr.Nod 2023,test1090,1,...,48126341.0,48126973.0,-111011690.0,+,MER41B,LTR,ERV1,1.0,634.0,0.0
209,7,148638603,148638604,T,C,Liver,EZH2,Dr.Nod 2023,test1092,1,...,148638579.0,148638663.0,-10500000.0,+,HY5,scRNA,scRNA,1.0,84.0,0.0
210,7,148638596,148638597,T,C,Liver,EZH2,Dr.Nod 2023,test1094,1,...,148638579.0,148638663.0,-10500000.0,+,HY5,scRNA,scRNA,1.0,84.0,0.0


In [53]:
# df_merge.to_pickle('../data/RepeatMasker/repeat_masker.pickle', compression='infer', protocol=5, storage_options=None)
res.to_pickle('../data/RepeatMasker/repeat_masker_test.pickle', compression='infer', protocol=5, storage_options=None)
# df_merge1 = pd.read_pickle('../data/RepeatMasker/repeat_masker.pickle')

In [42]:
df_merge1#['repClass'].value_counts()

,chr,start,end,allele,driver,chrom,itemsReturned,bin,swScore,milliDiv,...,genoEnd,genoLeft,strand,repName,repClass,repFamily,repStart,repEnd,repLeft,id
0,5,1295203,1295203,G/A,1,5,1.0,594.0,289.0,254.0,...,1295316.0,-179619944.0,+,(CGGGG)n,Simple_repeat,Simple_repeat,2.0,133.0,0.0,2
1,5,1295226,1295226,G/A,1,5,1.0,594.0,289.0,254.0,...,1295316.0,-179619944.0,+,(CGGGG)n,Simple_repeat,Simple_repeat,2.0,133.0,0.0,2
2,5,1295226,1295226,G/A,1,5,1.0,594.0,289.0,254.0,...,1295316.0,-179619944.0,+,(CGGGG)n,Simple_repeat,Simple_repeat,2.0,133.0,0.0,2
3,5,1295226,1295226,G/A,1,5,1.0,594.0,289.0,254.0,...,1295316.0,-179619944.0,+,(CGGGG)n,Simple_repeat,Simple_repeat,2.0,133.0,0.0,2
4,5,1295226,1295226,G/A,1,5,1.0,594.0,289.0,254.0,...,1295316.0,-179619944.0,+,(CGGGG)n,Simple_repeat,Simple_repeat,2.0,133.0,0.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,X,72254391,72254391,C/T,0,X,1.0,1136.0,27572.0,45.0,...,72257675.0,-83012885.0,-,L1PA3,LINE,L1,0.0,6155.0,1.0,4
213,3,180585989,180585989,C/T,0,3,1.0,1962.0,1597.0,174.0,...,180586453.0,-17435977.0,-,LTR1C,LTR,ERV1,0.0,656.0,1.0,2
214,19,30558759,30558759,T/C,0,19,1.0,818.0,1071.0,167.0,...,30559427.0,-28569556.0,+,(TCCA)n,Simple_repeat,Simple_repeat,1.0,706.0,0.0,1
215,16,62818734,62818734,C/T,0,16,1.0,1064.0,9249.0,235.0,...,62822231.0,-27532522.0,-,L1MB8,LINE,L1,-1.0,7037.0,3103.0,1
